# Confidence of transcription

Hack to visualize the confidence at each token

In [1]:
import os
import sys
import torch

# allow import from parent directory (we are not in a module)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import whisper

/Users/bradylaw/code/whisper/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Clear speech example

In [2]:
model = whisper.load_model("base.en")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("../tests/clear.m4a")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [3]:
# decode the audio
options = whisper.DecodingOptions(
    language='en',
    task='transcribe',
    fp16=False,  # needed for m1 macs?
    beam_size=None,  # use greedy decoder
)
result = whisper.decode(model, mel, options)

TypeError: __init__() got an unexpected keyword argument 'token_logprobs'

In [ ]:
probability = torch.exp(torch.tensor(result.avg_logprob)).item()  # convert log probability to probability
print(f'{probability=:.2%}')
print(f'{result.text=}')
print(f'{result.token_logprobs=}')

probability=78.82%
result.text="Hello, can you understand what I'm saying or is it hard to understand?"


AttributeError: 'DecodingResult' object has no attribute 'token_logprobs'

## Mumbling speech example

In [ ]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("../tests/mumble.m4a")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [ ]:
# decode the audio
options = whisper.DecodingOptions(
    language='en',
    fp16=False,  # needed for m1 macs?
    beam_size=None,  # use greedy decoder
)
result = whisper.decode(model, mel, options)

In [ ]:
probability = torch.exp(torch.tensor(result.avg_logprob)).item()  # convert log probability to probability
print(f'{probability=:.2%}')
print(f'{result.text=}')
print(f'{result.token_logprobs=}')

probability=75.59%
result.text="Hello, can you understand what I'm saying or is it hard to understand?"


AttributeError: 'DecodingResult' object has no attribute 'token_logprobs'